In [151]:
import pandas as pd

In [152]:
nombre_archivo = 'precios_semana_20200518.txt'
# este nombre tiene que tomarse de la ejecución del script; para desarrollarlo lo hardcodeamos
extension = nombre_archivo.split('.')[1]
# guarda la extension del archivo
ruta = './Datasets/'
# harcodeamos ruta relativa

In [153]:
codfecha = (nombre_archivo.split('.')[0])[-8:]

In [154]:
fecha_carga = codfecha[0:4]+'/'+codfecha[4:6]+'/'+codfecha[6:8]

In [155]:
dfprecio = pd.read_csv(str(ruta)+str(nombre_archivo), sep='|')

In [156]:
dfprecio.head()

,precio,producto_id,sucursal_id
0,29.90,0000000002288,2-1-009
1,32.90,0000000002288,2-1-037
2,36.90,0000000002288,2-1-090
3,39.90,0000000002288,2-3-247
4,499.99,0000000205870,9-1-430


In [157]:
dfprecio.describe(include='all')

,precio,producto_id,sucursal_id
count,413333.000000,415287,415287
unique,NaN,60744,141
top,NaN,7791293022567,49-1-2
freq,NaN,101,15128
mean,185.769875,NaN,NaN
std,376.163571,NaN,NaN
min,0.350000,NaN,NaN
25%,67.000000,NaN,NaN
50%,119.000000,NaN,NaN
75%,214.500000,NaN,NaN


In [158]:
def sacar_espacios(df):
    '''
    Quitar espacios vacíos de de ambos lados de las cols con strings.
    '''
    sacar_espacios = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(sacar_espacios)
sacar_espacios(dfprecio)

,precio,producto_id,sucursal_id
0,29.90,0000000002288,2-1-009
1,32.90,0000000002288,2-1-037
2,36.90,0000000002288,2-1-090
3,39.90,0000000002288,2-3-247
4,499.99,0000000205870,9-1-430
...,...,...,...
415288,149.99,9569753142128,25-1-1
415289,34.99,9795403001143,25-1-1
415290,312.50,9990385651922,5-1-4
415291,312.50,9990385651939,5-1-4


In [159]:
dfprecio['precio']=dfprecio['precio'].astype(float)

In [160]:
dfprecio[dfprecio['precio'].isnull()|dfprecio['producto_id'].isnull()|dfprecio['sucursal_id'].isnull()]

,precio,producto_id,sucursal_id
8656,NaN,0643131504438,65-1-328
8982,NaN,0655257738062,65-1-328
8983,NaN,0655257738246,65-1-328
18050,NaN,3014260014445,65-1-328
18082,NaN,3014260019723,65-1-328
...,...,...,...
409962,NaN,8004200128702,65-1-328
410003,NaN,8004200128801,65-1-328
410043,NaN,8004200128900,65-1-328
410071,NaN,8004200129006,65-1-328


In [161]:
dfprecio[dfprecio['producto_id'].isnull()]

,precio,producto_id,sucursal_id
375118,203.0,NaN,NaN
375157,NaN,NaN,NaN
375193,240.0,NaN,NaN
375278,NaN,NaN,NaN
375357,NaN,NaN,NaN
375397,NaN,NaN,NaN


In [162]:
#elimino registros sin codigo de producto.
#registros sin codigo no me van a servir para ningun analisis
lista_sin_cod_prod = dfprecio[dfprecio['sucursal_id'].isnull()].index.tolist()


In [163]:
dfprecio.drop(lista_sin_cod_prod, inplace=True)

In [164]:
dfprecio.info()
#observo que solo quedan nulos en la columna 'precio'
#para salvar los nulos de la columna precio, procederemos a imputar el precio promedio de dicho producto en el resto de las sucursales.
#pero primero debemos normalizar el contenido de ambas columnas 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415287 entries, 0 to 415292
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       413331 non-null  float64
 1   producto_id  415287 non-null  object 
 2   sucursal_id  415287 non-null  object 
dtypes: float64(1), object(2)
memory usage: 12.7+ MB


In [165]:
# comencemos con la columna producto_id
# creo una columna 'cant' que indique la cantidad de caracteres
dfprecio['cant']=dfprecio['producto_id'].apply(lambda x: len(str(x)))

In [166]:
dfprecio['cant'].unique()

array([13, 17, 18], dtype=int64)

In [167]:
dfprecio[dfprecio['cant']==18].head()

,precio,producto_id,sucursal_id,cant
10003,829.0,10-1-2300006000003,10-1-2,18
10004,59.0,10-1-2300049000008,10-1-2,18
10005,59.0,10-1-2300075000003,10-1-2,18
10006,79.0,10-1-2300075000003,10-1-9,18
10007,1274.0,10-1-2300175000002,10-1-2,18


In [168]:




dfprecio['id2']=dfprecio['producto_id'].apply(lambda x: x[-13:])
import re
dfprecio['caractid2']=dfprecio['id2'].apply(lambda x: re.findall('\D',x))


In [169]:
dfprecio['caractid2'].describe()
#observo que no hay caracteres no numericos en el nuevo id

count     415287
unique         1
top           []
freq      415287
Name: caractid2, dtype: object

In [170]:
dfprecio.head()

,precio,producto_id,sucursal_id,cant,id2,caractid2
0,29.90,0000000002288,2-1-009,13,0000000002288,[]
1,32.90,0000000002288,2-1-037,13,0000000002288,[]
2,36.90,0000000002288,2-1-090,13,0000000002288,[]
3,39.90,0000000002288,2-3-247,13,0000000002288,[]
4,499.99,0000000205870,9-1-430,13,0000000205870,[]


In [171]:

dfprecio.pop('cant')
dfprecio.pop('id2')


0         0000000002288
1         0000000002288
2         0000000002288
3         0000000002288
4         0000000205870
              ...      
415288    9569753142128
415289    9795403001143
415290    9990385651922
415291    9990385651939
415292    9990385651946
Name: id2, Length: 415287, dtype: object

In [172]:

dfprecio.pop('caractid2')


0         []
1         []
2         []
3         []
4         []
          ..
415288    []
415289    []
415290    []
415291    []
415292    []
Name: caractid2, Length: 415287, dtype: object

In [173]:

dfprecio.head()

,precio,producto_id,sucursal_id
0,29.90,0000000002288,2-1-009
1,32.90,0000000002288,2-1-037
2,36.90,0000000002288,2-1-090
3,39.90,0000000002288,2-3-247
4,499.99,0000000205870,9-1-430


In [174]:
#normalizo sucursal
dfprecio_aux=dfprecio['sucursal_id'].str.split('-', expand=True)

In [175]:
dfprecio_aux[0]=dfprecio_aux[0].astype(int)
dfprecio_aux[1]=dfprecio_aux[1].astype(int)
dfprecio_aux[2]=dfprecio_aux[2].astype(int)


In [176]:
dfprecio_aux.head()
dfprecio_aux.describe()

,0,1,2
count,415287.000000,415287.000000,415287.000000
mean,16.885164,1.626978,563.579724
std,14.897733,1.137520,1172.142063
min,1.000000,1.000000,1.000000
25%,9.000000,1.000000,8.000000
50%,10.000000,1.000000,112.000000
75%,16.000000,2.000000,655.000000
max,65.000000,6.000000,6004.000000


In [177]:
dfprecio_aux['sucursal_id']=dfprecio_aux[0].astype(str)+'-'+dfprecio_aux[1].astype(str)+'-'+dfprecio_aux[2].astype(str)

In [178]:
dfprecio_aux.head()

,0,1,2,sucursal_id
0,2,1,9,2-1-9
1,2,1,37,2-1-37
2,2,1,90,2-1-90
3,2,3,247,2-3-247
4,9,1,430,9-1-430


In [179]:
dfprecio['sucursal_id']=dfprecio_aux['sucursal_id']

In [180]:
dfprecio['fecha']=fecha_carga

In [181]:
dfprecio.head()

,precio,producto_id,sucursal_id,fecha
0,29.90,0000000002288,2-1-9,2020/05/18
1,32.90,0000000002288,2-1-37,2020/05/18
2,36.90,0000000002288,2-1-90,2020/05/18
3,39.90,0000000002288,2-3-247,2020/05/18
4,499.99,0000000205870,9-1-430,2020/05/18


In [182]:
dfprecio.describe()

,precio
count,413331.000000
mean,185.769702
std,376.164471
min,0.350000
25%,67.000000
50%,119.000000
75%,214.500000
max,35900.000000


In [183]:
dfprecio['precio']=dfprecio['precio'].replace('',0)
dfprecio['precio']=dfprecio['precio'].astype(float)
dfprecio.dropna(inplace=True)

In [184]:
dfprecio.describe()

,precio
count,413331.000000
mean,185.769702
std,376.164471
min,0.350000
25%,67.000000
50%,119.000000
75%,214.500000
max,35900.000000


In [185]:
dfprecio.rename(columns={'producto_id': 'Id_Producto', 'sucursal_id': 'Id_Sucursal'}, inplace=True)

In [186]:
dfprecio.insert(0, 'Id', 0)

In [187]:
#exportacion a csv de la tabla
#from encodings import utf_8


ruta_export = './Data_out/'
nombre_archivo_out = 'producto_precio_' + str(codfecha) + '.csv'
dfprecio.to_csv(ruta_export+nombre_archivo_out, index=False, encoding='utf-8-sig')